In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from torchvision import transforms
from PIL import Image
import os
import torch.nn.functional as F
from tqdm import tqdm
import torchvision
import random


device = torch.device('cpu')

# Step 1: Prepare your dataset
class SketchDataset(Dataset):
    def __init__(self, csv_file, image_root_dir, sketch_root_dir, transform=None):
        
        self.data_frame = pd.read_csv(csv_file)
        self.image_root_dir = image_root_dir
        self.sketch_root_dir = sketch_root_dir
        self.transform = transform
        self.num_sketches = 3  # Update with the actual number of sketches
        self.num_samples = len(self.data_frame)

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        # print("idx: ",idx)
        img_name = os.path.join(self.image_root_dir, self.data_frame.iloc[idx, 0] + '.jpg')
        sketch_idx = idx % self.num_sketches  # Cyclic indexing for sketches
        # print(sketch_idx)
        # sketch_name = os.path.join(self.sketch_root_dir, self.data_frame.iloc[idx, 0] + '_segmentation'+'.png')
        sketch_name = os.path.join(self.sketch_root_dir, f"sketch_{sketch_idx + 1}.png")
        image = Image.open(img_name).convert('RGB')
        sketch = Image.open(sketch_name).convert('RGB')

        label = torch.tensor(self.data_frame.iloc[idx, 1:], dtype=torch.float32)

        rand_idx= random.randint(0, self.num_samples-1)
        rand_label = torch.tensor(self.data_frame.iloc[rand_idx, 1:], dtype=torch.float32)

        if self.transform:
            image = self.transform(image)
            sketch = self.transform(sketch)
        
        return label, sketch, image,img_name, rand_label

transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
])

# Modify paths as needed
# test_dataset = SketchDataset(csv_file='/home/cvlab/Karan/A_3/Dataset_A4/Test_data/Test_labels.csv', 
#                              image_root_dir='/home/cvlab/Karan/A_3/Dataset_A4/Test_data/Test_images',
#                              sketch_root_dir='/home/cvlab/Karan/A_3/Dataset_A4/Test_data/Paired_test_sketch',
#                              transform=transform)
test_dataset = SketchDataset(csv_file='/home/cvlab/Karan/A_3/Dataset_A4/Test_data/Test_labels copy.csv', 
                             image_root_dir='/home/cvlab/Karan/A_3/Dataset_A4/Test_data/Test_images',
                             sketch_root_dir='/home/cvlab/Karan/A_3/Dataset_A4/Test_data/Unpaired_sketch',
                             transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

In [ ]:
class Block(nn.Module):
    def __init__(self, in_channels, out_channels, down=True, act="relu", use_dropout=False):
        super(Block, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 4, 2, 1, bias=False, padding_mode="reflect")
            if down
            else nn.ConvTranspose2d(in_channels, out_channels, 4, 2, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU() if act == "relu" else nn.LeakyReLU(0.2),
        )

        self.use_dropout = use_dropout
        self.dropout = nn.Dropout(0.5)
        self.down = down

    def forward(self, x):
        x = self.conv(x)
        return self.dropout(x) if self.use_dropout else x


class Generator(nn.Module):
    def __init__(self, in_channels=10, features=64): # 3 earlier
        super().__init__()
        self.initial_down = nn.Sequential(
            nn.Conv2d(in_channels, features, 4, 2, 1, padding_mode="reflect"),
            nn.LeakyReLU(0.2),
        )
        self.down1 = Block(features, features * 2, down=True, act="leaky", use_dropout=False)
        self.down2 = Block(
            features * 2, features * 4, down=True, act="leaky", use_dropout=False
        )
        self.down3 = Block(
            features * 4, features * 8, down=True, act="leaky", use_dropout=False
        )
        self.down4 = Block(
            features * 8, features * 8, down=True, act="leaky", use_dropout=False
        )
        self.down5 = Block(
            features * 8, features * 8, down=True, act="leaky", use_dropout=False
        )
        self.down6 = Block(
            features * 8, features * 8, down=True, act="leaky", use_dropout=False
        )
        self.bottleneck = nn.Sequential(
            nn.Conv2d(features * 8, features * 8, 4, 2, 1), nn.ReLU()
        )

        self.up1 = Block(features * 8, features * 8, down=False, act="relu", use_dropout=True)
        self.up2 = Block(
            features * 8 * 2, features * 8, down=False, act="relu", use_dropout=True
        )
        self.up3 = Block(
            features * 8 * 2, features * 8, down=False, act="relu", use_dropout=True
        )
        self.up4 = Block(
            features * 8 * 2, features * 8, down=False, act="relu", use_dropout=False
        )
        self.up5 = Block(
            features * 8 * 2, features * 4, down=False, act="relu", use_dropout=False
        )
        self.up6 = Block(
            features * 4 * 2, features * 2, down=False, act="relu", use_dropout=False
        )
        self.up7 = Block(features * 2 * 2, features, down=False, act="relu", use_dropout=False)
        self.final_up = nn.Sequential(
            nn.ConvTranspose2d(features * 2, 3, kernel_size=4, stride=2, padding=1),
            nn.Tanh(),
        )

    def forward(self, x):
        d1 = self.initial_down(x)
        d2 = self.down1(d1)
        d3 = self.down2(d2)
        d4 = self.down3(d3)
        d5 = self.down4(d4)
        d6 = self.down5(d5)
        d7 = self.down6(d6)
        bottleneck = self.bottleneck(d7)
        up1 = self.up1(bottleneck)
        up2 = self.up2(torch.cat([up1, d7], 1))
        up3 = self.up3(torch.cat([up2, d6], 1))
        up4 = self.up4(torch.cat([up3, d5], 1))
        up5 = self.up5(torch.cat([up4, d4], 1))
        up6 = self.up6(torch.cat([up5, d3], 1))
        up7 = self.up7(torch.cat([up6, d2], 1))
        last = self.final_up(torch.cat([up7, d1], 1))
        # print("last: ",last.shape)
        return last

In [ ]:
# Load the pre-trained model
model = Generator(in_channels=10, features=64)  # Replace YourModel with the actual model class
# model.load_state_dict(torch.load('/home/cvlab/Karan/A_3/logs/CGAN_all/best_model.pth'))
# model.load_state_dict(torch.load('A_3/A4_Codes/CGAN_new_data_paired/best_model.pth'))
model.load_state_dict(torch.load('/home/cvlab/Karan/A_3/A4_Codes/CGAN_new_data/best_model.pth'))
model.to(device)
model.eval()

In [ ]:
import torch
import torchvision.models as models

# Load pre-trained model
model1 = models.resnet18(pretrained=True)

# Freeze model parameters
for param in model1.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to match the number of classes
num_ftrs = model1.fc.in_features
model1.fc = nn.Linear(num_ftrs, 7)  # Assuming 7 classes

# Load the trained model state dictionary
model1.load_state_dict(torch.load('/home/cvlab/Karan/A_3/A4_Codes/model_weights_new.pth'))

# Set the model to evaluation mode
model1.eval()

In [ ]:
import torch
import numpy as np
import os
import csv
from torchvision.models import inception_v3
from scipy.stats import entropy
from torchvision import transforms
from torch.utils.data import DataLoader, Subset
from tqdm import tqdm
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F

# Create a directory to save the images if it doesn't exist
output_dir = 'Unpaired_final'
os.makedirs(output_dir, exist_ok=True)

# Open CSV file to save image names and labels
csv_file = open('Unpaired_Final.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Image_Name', 'Label'])

# Define a function to generate images from sketches and labels
def generate_images(real_gen):
    with torch.no_grad():
        generated_images = model(real_gen)
        # generated_images = torch.sigmoid(generated_images)  # If your model's output is not normalized
    return generated_images.to(device)

# Function to calculate the inception score
def calculate_inception_score(images, batch_size=32, splits=1):
    # Load Inception V3 model
    model = inception_v3(pretrained=True, transform_input=True, aux_logits=True).to(device)
    model.eval()

    # Define transformations
    preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((299, 299)),
    ])

    # Calculate activations for real images
    preds = []
    for batch_start in tqdm(range(0, len(images), batch_size), desc="Calculating activations"):
        batch_images = images[batch_start:batch_start+batch_size]
        batch_images = [preprocess(img) for img in batch_images]
        batch_images = torch.stack(batch_images)
        pred = model(batch_images).detach().cpu().numpy()
        preds.append(pred)
    preds = np.concatenate(preds)

    # Calculate Inception Score
    scores = []
    for i in range(splits):
        part = preds[i * (preds.shape[0] // splits): (i + 1) * (preds.shape[0] // splits), :]
        p_yx = np.exp(part) / np.exp(part).sum(1, keepdims=True)
        p_y = np.exp(np.mean(part, axis=0)) / np.exp(np.mean(part, axis=0)).sum()
        scores.append(entropy(p_yx.T) + entropy(p_y))
    return np.mean(scores), np.std(scores)

# Compare generated images with real images and compute Inception Score
inception_scores = []
correct = 0
total = 0
# Initialize a dictionary to keep track of sampled images for each label
sampled_images_per_label = {label: 0 for label in range(7)}
batch_counter = 0

for labels, sketches, real_images, img_names, rand_label in test_loader:
    labels, sketches, real_images = labels.to(device), sketches.to(device), real_images.to(device)

    img_size = 256
    embed_gen = nn.Embedding(7, 1).to(device)
    embedding_gen = embed_gen(labels.long())
    embedding_gen = embedding_gen.unsqueeze(3)
    upsampled_embedding_gen = F.interpolate(embedding_gen, size=(img_size, img_size), mode='nearest')
    real_gen = torch.cat([sketches, upsampled_embedding_gen], dim=1)

    generated_images = generate_images(real_gen)
    generated_images_numpy = generated_images.cpu().detach().numpy()
    real_images_numpy = real_images.cpu().detach().numpy()

    # Calculate Inception Score between generated and real images
    inception_score, _ = calculate_inception_score(np.concatenate([generated_images_numpy, real_images_numpy]))
    inception_scores.append(inception_score)

    # Save only 5 images from each batch, one sample for each label
    for i in range(len(real_images)):
        label = labels[i].nonzero().item()

        outputs = model1(generated_images)
        # Get the predicted classes
        _, predicted = torch.max(outputs, 1)
        # Convert one-hot encoded labels to class indices
        labels_indices = torch.argmax(labels, dim=1)
        # Update number of correctly classified samples
        correct += (predicted == labels_indices).sum().item()
        total += labels.size(0)

        generated_img = transforms.ToPILImage()(generated_images[i].cpu())
        real_img = transforms.ToPILImage()(real_images[i].cpu())
        sketch = transforms.ToPILImage()(sketches[i].cpu())

        # Display generated and real images side by side
        comparison_img = Image.new('RGB', (generated_img.width , generated_img.height))
        comparison_img.paste(generated_img)

        # Save the comparison image with a unique filename based on batches
        comparison_img_name = os.path.join(output_dir, f"comparison_batch{batch_counter}_label{label}_sample{sampled_images_per_label[label]}.jpg")
        comparison_img.save(comparison_img_name)
        
        # Write image name and label to CSV
        csv_writer.writerow([os.path.basename(comparison_img_name), label])
        sampled_images_per_label[label] += 1  # Increment the count for this label

# Close CSV file
csv_file.close()

# Calculate the accuracy over the entire test data
accuracy = correct / total
print(f"Test Accuracy: {accuracy:.4f}")

# Print average Inception Score
print("Average Inception Score:", np.mean(inception_scores))


In [ ]:
import torch
import numpy as np
from torchvision.models import inception_v3
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
from scipy.linalg import sqrtm

# Define a function to generate images from sketches and labels
def generate_images(real_gen):
    with torch.no_grad():
        generated_images = model(real_gen)
        generated_images = torch.sigmoid(generated_images)  # If your model's output is not normalized
    return generated_images.to(device)
# Function to calculate the Frechet Inception Distance (FID)
def calculate_fid(real_images, generated_images, batch_size=32):
    # Load Inception V3 model
    model = inception_v3(pretrained=True, transform_input=True, aux_logits=True).to(device)
    model.eval()

    # Define transformations
    preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((299, 299)),
        # transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ])

    # Calculate activations for real images
    real_activations = []
    for batch_start in tqdm(range(0, len(real_images), batch_size), desc="Calculating real activations"):
        batch_real_images = real_images[batch_start:batch_start+batch_size]
        batch_real_images = torch.stack([preprocess(img) for img in batch_real_images])
        
        # conv = nn.Conv2d(256, 3, kernel_size=1)
        # batch_real_images = conv(batch_real_images)

        batch_real_activations = model(batch_real_images).detach().cpu().numpy()
        real_activations.append(batch_real_activations)
    real_activations = np.concatenate(real_activations)

    # Calculate activations for generated images
    generated_activations = []
    for batch_start in tqdm(range(0, len(generated_images), batch_size), desc="Calculating generated activations"):
        batch_generated_images = generated_images[batch_start:batch_start+batch_size]
        batch_generated_images = torch.stack([preprocess(img) for img in batch_generated_images])

        conv = nn.Conv2d(256, 3, kernel_size=1)
        batch_generated_images = conv(batch_generated_images)


        batch_generated_activations = model(batch_generated_images).detach().cpu().numpy()
        generated_activations.append(batch_generated_activations)
    generated_activations = np.concatenate(generated_activations)

    # Calculate mean and covariance for real and generated activations
    mu_real = np.mean(real_activations, axis=0)
    mu_generated = np.mean(generated_activations, axis=0)
    sigma_real = np.cov(real_activations, rowvar=False)
    sigma_generated = np.cov(generated_activations, rowvar=False)

    # Calculate FID
    diff = mu_real - mu_generated
    cov_sqrt = sqrtm(sigma_real.dot(sigma_generated))
    if np.iscomplexobj(cov_sqrt):
        cov_sqrt = cov_sqrt.real
    fid = np.sum(diff**2) + np.trace(sigma_real + sigma_generated - 2 * cov_sqrt)
    return fid

# Compare generated images with real images and compute FID
fid_scores = []
for labels, sketches, real_images, img_names, rand_label in test_loader:
    labels, sketches, real_images = labels.to(device), sketches.to(device), real_images.to(device)

    img_size = 256
    embed_gen = nn.Embedding(7, 1).to(device)
    embedding_gen = embed_gen(labels.long())
    embedding_gen = embedding_gen.unsqueeze(3)
    upsampled_embedding_gen = F.interpolate(embedding_gen, size=(img_size,img_size), mode='nearest')
    real_gen = torch.cat([sketches, upsampled_embedding_gen], dim=1)

    generated_images = generate_images(real_gen)
    generated_images_numpy = generated_images.cpu().detach().numpy()
    real_images_numpy = real_images.cpu().detach().numpy()

    # Calculate FID between generated and real images
    fid_score = calculate_fid(real_images_numpy, generated_images_numpy)
    fid_scores.append(fid_score)

# Print average FID
print("Average FID:", np.mean(fid_scores))
